In [5]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
# on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
# oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\off_train_sameUser_id_test.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')

off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test['discount_rate']=test['Discount_rate'].copy()
test.index=np.arange(0,len(test),1)

In [143]:
indexs=off_train[(off_train['Coupon_id']=='null')&(off_train['Date']!='null')].index
ordinary=off_train.iloc[indexs]
off_train.drop(indexs,axis=0,inplace=True)
ordinary.index=np.arange(0,len(ordinary),1)
off_train.index=np.arange(0,len(off_train),1)
ordinary['month']=ordinary['Date'].apply(lambda x: int(str(x)[5])).copy()
off_train['month']=off_train['Date_received'].apply(lambda x: int(str(x)[5])).copy()

ftest=off_train[off_train['month']==6].copy()
ftest.drop(['Date','month'],axis=1,inplace=True)
ftest['discount_rate']=ftest['Discount_rate']
ftest.index=np.arange(0,len(ftest),1)
# ftesty=off_train[off_train['month']==6]['Date'].copy()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
ftest['Date_received']=ftest['Date_received'].apply(lambda x: str(x))

p_train=ftest[off_train['Date']!='null'].copy()#正样本
n_train=ftest[off_train['Date']=='null'].copy()#负样本
p_train.index=np.arange(0,len(p_train),1)
n_train.index=np.arange(0,len(n_train),1)
#转换为时间格式
p_train['Date_received_copy']=p_train['Date_received'].copy()
p_train['Date_received_copy']=pd.to_datetime(p_train['Date_received_copy'])
p_train['Date']=pd.to_datetime(p_train['Date'])

p_train['date']=(p_train['Date']-p_train['Date_received_copy']).astype('timedelta64[D]')
p_train.loc[p_train[p_train['date']<=15].index,'date']=1
p_train.loc[p_train[p_train['date']>15].index,'date']=0
p_train['Date']=p_train['date'].copy()
p_train.drop(['date','Date_received_copy'],axis=1,inplace=True)
p_train['Date']=p_train['Date'].astype(np.int64)
n_train['Date']=0
ftest=pd.concat([p_train,n_train],axis=0)
ftest=ftest.sort_values(by=['User_id'])
ftest.index=np.arange(0,len(ftest),1)

ftesty=pd.DataFrame()
ftesty['Date']=ftest['Date']
ftesty['Coupon_id']=ftest['Coupon_id']
ftesty.to_csv(r'D:\Data\TCForNewComer\new_script\ftesty.csv',index=False)

In [3]:
off_train['ord_buy']=0
off_train['same_merc']=0
test['ord_buy']=0
test['same_merc']=0
i=0
j=10
length=len(set(off_train['User_id'].values))
for uid in set(off_train['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end=';')
        j=j+10
        
    tt=off_train[off_train['User_id']==uid]
    tto=ordinary[ordinary['User_id']==uid]
    if len(tto)!=0:
        for mid in set(tt['Merchant_id'].values):
            tat=tt[tt['Merchant_id']==mid]
            tato=tto[tto['Merchant_id']==mid]
            if len(tato)!=0:
                off_train.loc[tat.index,'ord_buy']=len(tato)
    for cid in set(tt['Coupon_id'].values):
        tct=tt[tt['Coupon_id']==cid]
        for month in set(tct['month'].values):
            tctm=tct[tct['month']==month]
            off_train.loc[tctm.index,'same_merc']=len(tctm)
    i+=1
    
for uid in set(test['User_id'].values):
    if uid in off_train['User_id'].values:
        tt=test[test['User_id']==uid]
        for mid in set(tt['Merchant_id'].values):
            if mid in off_train[off_train['User_id']==uid]['Merchant_id'].values:
                test.loc[tt[tt['Merchant_id']==mid].index,'ord_buy']=off_train[(off_train['User_id']==uid)&(off_train['Merchant_id']==mid)]['ord_buy'].max()
    for cid in set(test[test['User_id']==uid]['Coupon_id'].values):
        tct=test[(test['Coupon_id']==cid)&(test['User_id']==uid)]
        test.loc[tct.index,'same_merc']=len(tct)
        
for uid in set(ftest['User_id'].values):
    if uid in off_train['User_id'].values:
        tt=ftest[ftest['User_id']==uid]
        for mid in set(tt['Merchant_id'].values):
            if mid in off_train[off_train['User_id']==uid]['Merchant_id'].values:
                ftest.loc[tt[tt['Merchant_id']==mid].index,'ord_buy']=off_train[(off_train['User_id']==uid)&(off_train['Merchant_id']==mid)]['ord_buy'].max()
    for cid in set(ftest[ftest['User_id']==uid]['Coupon_id'].values):
        tct=ftest[(ftest['Coupon_id']==cid)&(ftest['User_id']==uid)]
        ftest.loc[tct.index,'same_merc']=len(tct)

# ordinary.drop(['month'],axis=1,inplace=True)
# off_train.drop(['month'],axis=1,inplace=True)
off_train.to_csv(r'D:\Data\TCForNewComer\new_script\train_off_train.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv',index=False)
ftest.to_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv',index=False)
ftesty.to_csv(r'D:\Data\TCForNewComer\new_script\ftesty.csv',index=False)

. 10 %;.. 20 %;... 30 %;.... 40 %;..... 50 %;...... 60 %;....... 70 %;........ 80 %;......... 90 %;

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
off_train.to_csv(r'D:\Data\TCForNewComer\new_script\train_off_train.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv',index=False)
ftest.to_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv',index=False)
ftesty.to_csv(r'D:\Data\TCForNewComer\new_script\ftesty.csv',index=False)

In [189]:
import pandas as pd
import numpy as np
off_train=pd.read_csv(r'D:\Data\TCForNewComer\new_script\off_train.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\new_script\test.csv')
ftest=pd.read_csv(r'D:\Data\TCForNewComer\new_script\ftest.csv')

In [2]:
off_train['Coupon_month_num']=0             #每月领取张数
# off_train['Coupon_month_nouse_num']=0       #每月领取没有使用张数
# off_train['Coupon_month_use_num']=0         #每月使用张数
# off_train['Coupon_month_use_prob']=0        #每月使用概率
off_train['month']=off_train['Date_received'].apply(lambda x: int(str(x)[5])).copy()
i=0
j=10
length=len(set(off_train['User_id'].values))
for uid in set(off_train['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end=';')
        j=j+10
    
    tt=off_train[off_train['User_id']==uid]
    for month in set(tt['month'].values):
        tat=tt[tt['month']==month]
        off_train.loc[tat.index,'Coupon_month_num']=len(tat)
    i=i+1
#         off_train.loc[tat.index,'Coupon_month_nouse_num']=len(tat[tat['Date']=='null'])
#         off_train.loc[tat.index,'Coupon_month_use_num']=len(tat[tat['Date']!='null'])
#         off_train.loc[tat.index,'Coupon_month_use_prob']=len(tat[tat['Date']=='null'])
Test=[test,ftest]
for data in Test:
    data['Coupon_month_num']=0
    for uid in set(data['User_id'].values):
        tt=data[data['User_id']==uid]
        data.loc[tt.index,'Coupon_month_num']=len(tt)

. 10 %;.. 20 %;... 30 %;.... 40 %;..... 50 %;...... 60 %;....... 70 %;........ 80 %;......... 90 %;

In [92]:
#对正负样本进行赋值
combine=[off_train,test,ftest]
for data in combine:
    data['Date_received']=data['Date_received'].apply(lambda x: str(x))

p_train=off_train[off_train['Date']!='null'].copy()#正样本
n_train=off_train[off_train['Date']=='null'].copy()#负样本
p_train.index=np.arange(0,len(p_train),1)
n_train.index=np.arange(0,len(n_train),1)
#转换为时间格式
p_train['Date_received_copy']=p_train['Date_received'].copy()
p_train['Date_received_copy']=pd.to_datetime(p_train['Date_received_copy'])
p_train['Date']=pd.to_datetime(p_train['Date'])

p_train['date']=(p_train['Date']-p_train['Date_received_copy']).astype('timedelta64[D]')
p_train.loc[p_train[p_train['date']<=15].index,'date']=1
p_train.loc[p_train[p_train['date']>15].index,'date']=0
p_train['Date']=p_train['date'].copy()
p_train.drop(['date','Date_received_copy'],axis=1,inplace=True)
p_train['Date']=p_train['Date'].astype(np.int64)
#转换为星期几x.weekday()+1
test['Date_received']=(test['Date_received'].apply(lambda x: int(x[6:]))).copy()
ftest['Date_received']=(ftest['Date_received'].apply(lambda x: int(x[6:]))).copy()
#正样本为1，负样本为0
n_train['Date']=0
off_train=pd.concat([p_train,n_train],axis=0)
off_train['Date_received']=(off_train['Date_received'].apply(lambda x: int(x[6:]))).copy()
off_train['Date_received']=off_train['Date_received'].astype(np.int64)

off_train['discount_rate']=off_train['Discount_rate'].copy()#增加一列可以看出实际商品价格
#将日期转化为多少号
combine=[off_train,test,ftest]
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return float(x.split(':')[0])
for data in combine:
    data.index=np.arange(0,len(data),1)
    data['discount_rate']=data['discount_rate'].apply(Discount_rate)
    
off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)

In [98]:
#转换优惠率
combine=[off_train,test,ftest]
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return float(x.split(':')[0])/float(x.split(':')[1])
for data in combine:
    data.index=np.arange(0,len(data),1)
    data['Discount_rate']=data['Discount_rate'].apply(Discount_rate)

In [ ]:
#填充Distance列的空缺值
strr=off_train[off_train['Distance']!='null']
null=off_train[off_train['Distance']=='null']
strr.index=np.arange(0,len(strr),1)
null.index=np.arange(0,len(null),1)
strr['Distance']=strr['Distance'].apply(lambda x: int(x))
print(strr.info())
for uid in set(null['User_id'].values):
    tt=strr[strr['User_id']==uid]
    if len(tt)==0:
        null.loc[null[null['User_id']==uid].index,'Distance']=0
    else:
        null.loc[null[null['User_id']==uid].index,'Distance']=tt['Distance'].median()
off_train=pd.concat([strr,null],axis=0)
off_train.index=np.arange(0,len(off_train),1)

for i in ftest[ftest['Distance']=='null'].index:
    ftest.loc[i,'Distance']=int(off_train[(off_train['User_id']==ftest.loc[i,'User_id'])&(off_train['Merchant_id']==ftest.loc[i,'Merchant_id'])]['Distance'].max())
    
#off_train\test\ftest
for uid in (test[test['Distance']=='null']['User_id'].values):
    if uid in off_train['User_id'].values:
        test.loc[test[(test['Distance']=='null')&(test['User_id']==uid)].index,\
                 'Distance']=off_train[off_train['User_id']==uid]['Distance'].median()
    else:
        test.loc[test[(test['Distance']=='null')&(test['User_id']==uid)].index,'Distance']=0

combine=[off_train,test,ftest]
for data in combine:
    data['Distance']=data['Distance'].apply(lambda x: int(x))

In [18]:
#找相同uid或mid的行并输出
import pandas as pd
import numpy as np
off_train=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_off_train.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv')
ftest=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv')
cols=['User_id','Merchant_id']
def set_same(col):
    b=set(test[col].values)
    c=set(off_train[col].values)
    print(col,': ','test ',len(b))
    print('off_train ',len(c))
    print(len(b&c))
    return (b&c)

#保留off_train中的和test的User_id或mid一样的行
row=[]
for col in cols:
    i=0
    j=10
    sett=set_same(col)
    length=len(sett)
    for inde in (sett):
        rate=int(i*100/length)
        if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
            print('.'*int(rate/10),rate,'%',end=':')
            j=j+10
        row.extend(list(off_train[off_train[col]==inde].index))
        i+=1
    
row=list(set(row))
temp=off_train.loc[row]

temp=temp.drop(list(temp[temp['Date']==1].index),axis=0)
date=off_train[off_train['Date']==1]
off_train_same_umid=pd.concat([temp,date],axis=0)
off_train_same_umid=off_train_same_umid.sort_values(by=['User_id'])
off_train_same_umid.index=np.arange(0,len(off_train_same_umid),1)
off_train_same_umid.to_csv(r'D:\Data\TCForNewComer\new_script\sameumid\off_train_same_umid.csv',index=False)

User_id :  test  76309
off_train  510698
72149
. 10 %:.. 20 %:... 30 %:.... 40 %:..... 50 %:...... 60 %:....... 70 %:........ 80 %:......... 90 %:Merchant_id :  test  1559
off_train  5599
1474
. 10 %:.. 20 %:... 30 %:.... 40 %:..... 50 %:...... 60 %:....... 70 %:........ 80 %:......... 90 %:

### 其中discount_rate代表商品实际的价格

In [7]:
import pandas as pd
import numpy as np
import time
off_train_same_umid=pd.read_csv(r'D:\Data\TCForNewComer\new_script\sameumid\off_train_same_umid.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv')
ftest=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv')

In [4]:
off_train_same_umid.head(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,ord_buy,same_merc,Coupon_month_num,month,discount_rate,Courate_month_l6,Courate_month_m6,Coupon_month_getl30,Coupon_month_getm30,Coupre_month_usepro,Coupon_premonth_usepro,Coupon_premonth_get
0,4,1433,8735,6.00,10,14,0,0,1,11525,2,30.00,1,0,1,0,0.005083,0.0,0.0
1,4,1469,2902,0.95,10,7,0,0,1,3102,6,0.95,1,0,1,0,0.028666,0.0,0.0
2,36,1041,13490,6.00,4,25,0,0,1,6815,1,30.00,2,0,2,0,0.035070,0.0,0.0
3,36,5717,12349,4.00,8,25,0,0,1,4921,1,20.00,2,0,2,0,0.000000,0.0,0.0
4,144,1553,10027,0.95,1,27,0,0,1,19,2,0.95,1,0,1,0,0.050000,0.0,0.0
5,215,129,8944,6.00,1,24,0,1,1,319,5,30.00,1,0,1,0,0.000000,0.0,0.0
6,236,1041,13490,6.00,0,25,0,0,1,6815,1,30.00,2,0,2,0,0.035070,0.0,2.0
7,236,1433,8735,6.00,0,11,0,0,1,11525,2,30.00,1,0,1,0,0.005083,0.0,2.0
8,236,5717,12349,4.00,0,25,0,0,1,4921,1,20.00,2,0,2,0,0.000000,0.0,2.0
9,238,760,2418,6.00,4,8,0,0,1,29285,6,30.00,1,0,1,0,0.390655,0.0,0.0


In [ ]:
for i in set(off_train_same_umid['discount_rate'].values):
    print(i,len(off_train_same_umid[off_train_same_umid['discount_rate']==i]))

In [4]:
#每月领取的优惠券张数
combine=[off_train_same_umid]
for data in combine:
    i=0
    j=10
    length=len(set(data['User_id'].values))
    
    data['Courate_month_l6']=0              #每月领取优惠券大于Discount_rate6的张数
    data['Courate_month_m6']=0              #每月领取优惠券小于等于Discount_rate6的张数
    data['Coupon_month_getl30']=0            #每月领取优惠券小于等于discount_rate30的张数
    data['Coupon_month_getm30']=0           #每月领取优惠券大于discount_rate30的张数
    for uid in set(data['User_id'].values):
        rate=int(i*100/length)
        if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
            print('.'*int(rate/10),rate,'%',end=':')
            j=j+10
            
        tt=data[data['User_id']==uid]
        for month in np.arange(1,7,1):
            tat=tt[tt['month']==month]
            if len(tat)==0:
                continue
            else:
                data.loc[tat.index,'Courate_month_l6']=len(tat[tat['Discount_rate']<=6])
                data.loc[tat.index,'Courate_month_m6']=len(tat[tat['Discount_rate']>6])
                data.loc[tat.index,'Coupon_month_getl30']=len(tat[tat['discount_rate']<=30])
                data.loc[tat.index,'Coupon_month_getm30']=len(tat[tat['discount_rate']>30])
        i=i+1

combine=[test,ftest]
for data in combine:
    data['Courate_month_l6']=0              #每月领取优惠券大于Discount_rate6的张数
    data['Courate_month_m6']=0              #每月领取优惠券小于等于Discount_rate6的张数
    data['Coupon_month_getl30']=0            #每月领取优惠券小于等于discount_rate30的张数
    data['Coupon_month_getm30']=0           #每月领取优惠券大于discount_rate30的张数
    for uid in set(data['User_id'].values):
        tt=data[data['User_id']==uid]
        data.loc[tt.index,'Courate_month_l6']=len(tt[tt['Discount_rate']<=6])
        data.loc[tt.index,'Courate_month_m6']=len(tt[tt['Discount_rate']>6])
        data.loc[tt.index,'Coupon_month_getl30']=len(tt[tt['discount_rate']<=30])
        data.loc[tt.index,'Coupon_month_getm30']=len(tt[tt['discount_rate']>30])

. 10 %:.. 20 %:... 30 %:.... 40 %:..... 50 %:...... 60 %:....... 70 %:........ 80 %:......... 90 %:

In [9]:
off_train_same_umid['Coupon_month_num']=0 #该优惠券本月发放的数目
off_train_same_umid['Coupre_month_usepro']=0#该优惠券上个月使用的概率
i=0
j=10
length=len(set(off_train_same_umid['Merchant_id'].values))
for mid in set(off_train_same_umid['Merchant_id'].values):
    rate=int(i*100/length)
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end=':')
        j=j+10
            
    use=mid
    for month in np.arange(1,7,1):
        tt=off_train_same_umid[(off_train_same_umid['Merchant_id']==use)&(off_train_same_umid['month']==month)]
        if len(tt)!=0:
            off_train_same_umid.loc[tt.index,'Coupon_month_num']=len(tt)
            prett=off_train_same_umid[(off_train_same_umid['Merchant_id']==use)&(off_train_same_umid['month']==(month-1))]
            if len(prett)!=0:
                off_train_same_umid.loc[tt.index,'Coupre_month_usepro']=len(prett[prett['Date']==1])/len(prett)
    i=i+1

for mid in set(off_train_same_umid[off_train_same_umid['month']==1]['Merchant_id'].values):   #填充一月份的数据
    tt=off_train_same_umid[(off_train_same_umid['month']==1)&(off_train_same_umid['Merchant_id']==mid)]
    tat=off_train_same_umid[(off_train_same_umid['month']!=1)&(off_train_same_umid['Merchant_id']==mid)]
    if len(tat)!=0:
        off_train_same_umid.loc[tt.index,'Coupre_month_usepro']=tat['Coupre_month_usepro'].median()

combine=[ftest,test]
i=5
for data in combine:
    data['Coupon_month_num']=0 #该优惠券本月发放的数目
    data['Coupre_month_usepro']=0#该优惠券上个月使用的概率
    val=off_train_same_umid['Merchant_id'].values
    for mid in set(data['Merchant_id'].values):
        tt=data[data['Merchant_id']==mid]
        data.loc[tt.index,'Coupon_month_num']=len(tt)
        if mid in val:
            tat=off_train_same_umid[(off_train_same_umid['Merchant_id']==mid)&(off_train_same_umid['month']==i)]
            if len(tat)!=0:
                data.loc[tt.index,'Coupre_month_usepro']=len(tat[tat['Date']==1])/len(tat)
    i=6
#----------------------------------------------------------------------------------------------------------------------------------------------
off_train_same_umid['Coupon_premonth_usepro']=0  #用户上个月使用的概率
off_train_same_umid['Coupon_premonth_get']=0  #用户上个月领取的张数
i=0
j=10
length=len(set(off_train_same_umid['User_id'].values))
for uid in set(off_train_same_umid['User_id'].values):
    rate=int(i*100/length)
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end=':')
        j=j+10
    
    for month in np.arange(2,7,1):
        tt=off_train_same_umid[(off_train_same_umid['User_id']==uid)&(off_train_same_umid['month']==month)]
        tat=off_train_same_umid[(off_train_same_umid['User_id']==uid)&(off_train_same_umid['month']==(month-1))]
        if len(tt)!=0:
            off_train_same_umid.loc[tt.index,'Coupon_premonth_get']=len(tat)
            if len(tat)!=0:
                off_train_same_umid.loc[tt.index,'Coupon_premonth_usepro']=len(tat[tat['Date']==1])/len(tat)
    i=i+1

for uid in set(off_train_same_umid[off_train_same_umid['month']==1]['User_id'].values):   #填充一月份的数据
    tt=off_train_same_umid[(off_train_same_umid['month']==1)&(off_train_same_umid['User_id']==uid)]
    tat=off_train_same_umid[(off_train_same_umid['month']!=1)&(off_train_same_umid['User_id']==uid)]
    if len(tat)!=0:
        off_train_same_umid.loc[tt.index,'Coupon_premonth_get']=tat['Coupon_premonth_get'].median()
        off_train_same_umid.loc[tt.index,'Coupon_premonth_usepro']=tat['Coupon_premonth_usepro'].median()

combine=[ftest,test]
i=5
for data in combine:
    data['Coupon_premonth_usepro']=0  #用户上个月使用的概率
    data['Coupon_premonth_get']=0  #用户上个月领取的张数
    val=off_train_same_umid['User_id'].values
    for uid in set(data['User_id'].values):
        tt=data[data['User_id']==uid]
        if uid in val:
            tat=off_train_same_umid[(off_train_same_umid['User_id']==uid)&(off_train_same_umid['month']==i)]
            data.loc[tt.index,'Coupon_premonth_get']=len(tat)
            if len(tat)!=0:
                data.loc[tt.index,'Coupre_month_usepro']=len(tat[tat['Date']==1])/len(tat)
    i=6

. 10 %:.. 20 %:... 30 %:.... 40 %:..... 50 %:...... 60 %:....... 70 %:........ 80 %:......... 90 %:. 10 %:.. 20 %:... 30 %:.... 40 %:..... 50 %:...... 60 %:....... 70 %:........ 80 %:......... 90 %:

In [42]:
off_train_same_umid.to_csv(r'D:\Data\TCForNewComer\new_script\sameumid\off_train_same_umid.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv',index=False)
ftest.to_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv',index=False)

In [38]:
import pandas as pd
import numpy as np
import time
off_train_same_umid=pd.read_csv(r'D:\Data\TCForNewComer\new_script\sameumid\off_train_same_umid.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv')
ftest=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_ftest.csv')
# on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
# on_train[on_train[on_train['Coupon_id']=='fixed'].index,'Coupon_id']='10'

# col=['online_buy','online_cou','low_price','click']
# combine=[off_train_same_umid,test,ftest]
# for data in combine:
#     for i in col:
#         data[i]=0

# for data in combine:
#     for uid in set(data['User_id'].values):
        

In [ ]:
set(off_train_same_umid['Coupon_premonth_get'].values)

In [7]:
import pandas as pd
import numpy as np
off_train_same_umid=pd.read_csv(r'D:\Data\TCForNewComer\new_script\sameumid\off_train_same_umid.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\new_script\train_test.csv')
#正负样本取样
def set_same(fir,sec,col):
    b=set(fir[col].values)
    c=set(sec[col].values)
    print(col,': ',len(b))
    print(len(c))
    print(len(c&b))
    return (c&b)
posi=off_train_same_umid[off_train_same_umid['Date']==1]
negt=off_train_same_umid[off_train_same_umid['Date']==0]
# negt=negt.drop_duplicates(subset=['Merchant_id','Coupon_id','Discount_rate','Distance','Date','Date_received','month'])
print(len(negt))
print(len(posi))
negt=negt.drop_duplicates(subset=['User_id','discount_rate'])
# negt=negt.drop_duplicates(subset=['Merchant_id','Coupon_id','Distance'])
print('len:',len(negt))
negt.index=np.arange(0,len(negt),1)
index1=[]
for uid in set_same(test,negt,'User_id'):
    for mid in set_same(test,negt,'Merchant_id'):
    index1.extend(negt[negt['User_id']==uid].index)
temp1=negt.loc[index1]
posi=pd.concat([posi,temp1],axis=0)
posi.index=np.arange(0,len(posi),1)
print(len(posi))
negt.drop(index1,axis=0,inplace=True)
negt.index=np.arange(0,len(negt),1)      
posi.to_csv(r'D:\Data\TCForNewComer\new_script\sameumid\sampling.csv',index=False)

566752
64395
len: 428692
User_id :  76309
312451
70531
194390


In [ ]:
posi=off_train_same_umid[off_train_same_umid['Date']==1]
negt=off_train_same_umid[off_train_same_umid['Date']==0]
# negt=negt.drop_duplicates(subset=['User_id','Discount_rate','Date'])
negt=negt.drop_duplicates(subset=['Merchant_id','Coupon_id','Discount_rate','Distance','Date','Date_received','month'])

In [9]:
len(negt)

298697

In [10]:
len(posi)

194390

In [4]:
negt[negt['User_id']==947]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,ord_buy,same_merc,Coupon_month_num,month,discount_rate,Courate_month_l6,Courate_month_m6,Coupon_month_getl30,Coupon_month_getm30,Coupre_month_usepro,Coupon_premonth_usepro,Coupon_premonth_get
40,947,3939,1110,20.0,10,31,0,0,1,1434,5,100.0,1,1,0,2,0.181818,0.0,0.0
41,947,6610,12941,5.0,7,6,0,1,1,44,5,150.0,1,1,0,2,0.000000,0.0,0.0
42,947,8506,12414,20.0,7,10,0,1,2,5,6,200.0,0,2,0,2,0.333333,0.0,2.0


In [ ]:
posi.sort_values(by=['User_id','Date'])